In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import mean_squared_error

from data_loader import *
from stats import *
from train_test import *

## Settings

In [ ]:
seed = 7
num_train = 60
num_test = 20
num_total = num_train + num_test

window_size = 10
kernel_size = 5

tickers = [
    "AMZN", "AAPL", "NVDA", "GME", "MSFT", "META", "AVGO", "NFLX", "GOOG",
    "AMGN", "JNJ", "UNH", "VKTX", "NVAX", "OCGN", "ABBV", "REGN", "ELAN", 
    "AAL", "LUV", "JEPI", "SOFI", "ORCL", "PLTR", 
    # "TSLA", "SOUN", "NIO", "MDB"
]

# tickers = [
#     "AAPL", "MSFT", "AMZN", "TSLA", "NVDA", "META", "GOOGL", "GOOG", "JPM", 
#     "V", "UNH", "JNJ", "WMT", "PG", "MA", "HD", "BAC", "DIS", "PYPL", 
#     "NFLX", "KO", "PFE", "CSCO", "XOM", "VZ", "INTC", "T", "MRK", "CVX", 
#     "NKE", "ORCL", "ABT", "CMCSA",  "MDT", "MCD", "WFC", "LLY", "HON", 
#     "QCOM", "AMGN", "TXN", "SBUX", "MMM", "DHR", "BMY", "C", "GS", "CAT", 
#     "SPG", "TMO", "LOW", "UPS", "MS", "UNP", "RTX", "AXP", "BLK", "SCHW", 
#     "BKNG", "AMT", "F", "DE", "LIN", "ADP", "PLD", "ISRG", "SYK", "GILD", 
#     "LMT", "MO", "CL", "MDLZ", "TGT", "USB", "SO", "NEE", "AON", "MMC", 
#     "CB", "PGR", "CI", "MET", "AFL", "PRU", "ALL", "TRV", "AIG", "HIG", 
#     "LNC", "PFG", "UNM", "L", "CNA", "CINF", "AIZ"
# ]

#deciding whether to send to the cpu or not if available
if torch.cuda.is_available() and try_cuda:
    cuda = True
    torch.cuda.mnaual_seed(seed)
else:
    cuda = False
    torch.manual_seed(seed)


writer = SummaryWriter(log_dir=f"runs/sparsely_connected/{datetime.now().strftime('%Y%m%d_%H%M%S')}")


## Data

In [ ]:
X = []
y = []

for ticker in tickers:
    X1, y1 = load_stock(ticker)
    X.append(X1)
    y.append(y1)

X = np.stack(X)
X = X.reshape((X.shape[0] * X.shape[1], X.shape[-1]))
y = np.stack(y)


X_train, y_train = setup_cov_tensors(X[:, -num_total: -num_test], y[:, -num_total: -num_test], window_size)
X_test, y_test = setup_cov_tensors(X[:, -num_test-window_size:], y[:, -num_test-window_size:], window_size)


In [ ]:
print("Input", X_train.shape)
print("Out", y_train.shape)

## Model

In [ ]:
# Define the 1D convolutional model
class FullyConnectedStockReturnPredictor(nn.Module):
    def __init__(self, neteork_in_size, neteork_out_size, kernel_size, sequence_length):
        super(FullyConnectedStockReturnPredictor, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=neteork_in_size, out_channels=30, kernel_size=kernel_size)
        self.conv2 = nn.Conv1d(in_channels=30, out_channels=10, kernel_size=kernel_size)
        num_conv_layers = 2
        self.fc1 = nn.Linear(10 * (sequence_length - num_conv_layers * kernel_size + num_conv_layers * 1), neteork_out_size)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.tanh(x)
        x = self.conv2(x)
        x = torch.tanh(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x


## Train

In [ ]:
# Instantiate the model, define loss and optimizer
model = FullyConnectedStockReturnPredictor(X.shape[0], y.shape[0], kernel_size, window_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

for epoch in range(4000):
    train(epoch, X_train, y_train, model, optimizer, criterion, writer)
    if epoch % 100 == 0:
        pred, obj = test(epoch, X_test, y_test, model, writer)

writer.close()


## Plot

In [ ]:
df = compute_prediction_stats(tickers, pred, obj)
df

In [ ]:
df.to_csv("fully_connected_stock.df", index=False)

In [ ]:
stock_id = 1
_pred = pred[:, stock_id]
_obj = obj[:, stock_id]

obj_sort = np.argsort(_obj)
plt.plot(np.cumsum(_pred[obj_sort]), label="pred")
plt.plot(np.cumsum(_obj[obj_sort]), label="obj")

plt.legend()

In [ ]:
plt.plot((_pred), label="pred")
plt.plot((_obj), label="obj")

plt.legend()